### Imports

In [1]:
from __future__ import division, print_function
from collections import OrderedDict

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt
import time
import os
import cv2
from PIL import Image, ImageChops

plt.ion()   # interactive mode
%matplotlib inline
%env TMP="/tmp"
%env JOBLIB_TEMP_FOLDER="/tmp"

env: TMP="/tmp"
env: JOBLIB_TEMP_FOLDER="/tmp"


In [2]:
def summary(input_size, model):
    def register_hook(module):
        def hook(module, input, output):
            class_name = str(module.__class__).split('.')[-1].split("'")[0]
            module_idx = len(summary)

            m_key = '%s-%i' % (class_name, module_idx+1)
            summary[m_key] = OrderedDict()
            summary[m_key]['input_shape'] = list(input[0].size())
            summary[m_key]['input_shape'][0] = -1
            summary[m_key]['output_shape'] = list(output.size())
            summary[m_key]['output_shape'][0] = -1

            params = 0
            if hasattr(module, 'weight'):
                params += torch.prod(torch.LongTensor(list(module.weight.size())))
                if module.weight.requires_grad:
                    summary[m_key]['trainable'] = True
                else:
                    summary[m_key]['trainable'] = False
            if hasattr(module, 'bias'):
                params +=  torch.prod(torch.LongTensor(list(module.bias.size())))
            summary[m_key]['nb_params'] = params

        if not isinstance(module, nn.Sequential) and \
           not isinstance(module, nn.ModuleList) and \
           not (module == model):
            hooks.append(module.register_forward_hook(hook))

    # check if there are multiple inputs to the network
    if isinstance(input_size[0], (list, tuple)):
        x = [Variable(torch.rand(1,*in_size)) for in_size in input_size]
    else:
        x = Variable(torch.rand(1,*input_size))

    # create properties
    summary = OrderedDict()
    hooks = []
    # register hook
    model.apply(register_hook)
    # make a forward pass
    model(x)
    # remove these hooks
    for h in hooks:
        h.remove()

    return summary

### Data

In [3]:
TRAIN_DIR = '../data/boneage-training-dataset/'
TRAIN_LABEL = '../data/train.csv'
FILETYPE = '.png'
use_gpu = torch.cuda.is_available()

### Dataset class

In [4]:
class BoneAgeDataset(Dataset):
    """Bone Age dataset."""

    def __init__(self, root_dir, labels, filetype, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.labels = labels
        self.root_dir = root_dir
        self.transform = transform
        self.filetype = filetype

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir, str(self.labels.iloc[idx]['id']) + self.filetype)
        img = Image.open(img_name).convert('RGB')
#         image = cv2.imread(img_name)[:,:,::-1]
        label = self.labels.iloc[idx]['boneage'].astype(np.float32)

        if self.transform:
            img = self.transform(img)
            
        sample = {'image': img, 'label': label}

        return sample

### Transforms

In [13]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Scale(1024),
        transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Scale(1024),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]),
}

### Data loaders

In [14]:
labels = pd.read_csv(TRAIN_LABEL)
bins = xrange(20)
labels['bins'] = np.digitize(labels['boneage'], bins=bins)

In [15]:
skf = StratifiedKFold(n_splits=4)
splits = []
for train, test in skf.split(np.zeros(len(labels)), labels['bins']):
    fold_label = {
        'train': labels.loc[train, ('id', 'boneage')],
        'val': labels.loc[test, ('id', 'boneage')]
    }
    splits.append(fold_label)

In [16]:
fold_label = splits[0]

In [17]:
image_datasets = {x: BoneAgeDataset(TRAIN_DIR, fold_label[x], FILETYPE, data_transforms[x])
                  for x in ['train', 'val']}
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8, shuffle=True, num_workers=6, pin_memory=True)
              for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

In [18]:
for i_batch, sample in enumerate(dataloders['val']):
    print(i_batch, sample['image'].size())
    print(i_batch, sample['image'].mean())
    if i_batch == 2:
        break

0 torch.Size([8, 3, 224, 224])
0 -1.09920149019
1 torch.Size([8, 3, 224, 224])
1 -0.755285097422
2 torch.Size([8, 3, 224, 224])
2 -0.851031367439


In [27]:
class Net(nn.Module):
    def __init__(self, input_shape=(3, 224, 224)):
        super(Net, self).__init__()
        
        resnet = models.resnet50(pretrained=True)
        num_ftrs = resnet.fc.in_features
        resnet.fc = nn.Linear(num_ftrs, 1)

        self.resnet = resnet

    def forward(self, x):
        x = self.resnet(x)
        return x
    
model = Net([3, 224, 224])
    
# print(summary([3, 224, 224], model))

if use_gpu:
    model.cuda()

In [28]:
lr = 0.01
momentum = 0.9
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [29]:
def ccc(x, y):
#     print('X:', x)
    mean_x = torch.mean(x)
#     print('Mean x:', mean_x)
    mean_y = torch.mean(y)
#     print('Mean y:', mean_y)
    xm = x.sub(mean_x)
    ym = y.sub(mean_y)
    xn = torch.norm(xm, 2)
#     print('Xn:', xn)
    yn = torch.norm(ym, 2)
#     print('Yn:', yn)
    xydot = ym.dot(xm.view(-1))
#     print('XYdot:', xydot)

    ccc_num = 2 * xydot / xm.size()[0]
#     print('CCC num:', ccc_num)
    ccc_den = xn + yn + torch.pow(mean_x.sub(mean_y), 2)
#     print('CCC den:', ccc_den)

    ccc = ccc_num / ccc_den
    return ccc
    

def train(epoch, train_loader):
    model.train()
    for batch_idx, sample in enumerate(train_loader):
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.l1_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % (1000 // len(target)) == 0:
            c = ccc(output, target)
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tCCC: {:.4f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0], c.data.cpu().numpy()[0]))

def test(test_loader):
    model.eval()
    test_loss = 0
    for sample in test_loader:
        data = sample['image']
        target = sample['label'].float()
        if use_gpu:
            data, target = data.cuda(async=True), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        loss = F.l1_loss(output, target)
        test_loss += loss.data[0]
        c = ccc(output, target)

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, CCC: {}\n'.format(test_loss, c.data.cpu().numpy()[0]))

In [30]:
for epoch in range(1, 10 + 1):
    for phase in ['train', 'val']:
        if phase == 'train':
            train(epoch, dataloders[phase])
        if phase == 'val':
            test(dataloders[phase])

Train Epoch: 1 [0/9453 (0%)]	Loss: 149.414124	CCC: -0.0000

Test set: Average loss: 4.7841, CCC: -6.47708368301

Train Epoch: 2 [0/9453 (0%)]	Loss: 28.835466	CCC: -0.6633

Test set: Average loss: 4.9129, CCC: 0.755845487118

Train Epoch: 3 [0/9453 (0%)]	Loss: 43.001381	CCC: 0.4561

Test set: Average loss: 5.1035, CCC: 4.12177610397

Train Epoch: 4 [0/9453 (0%)]	Loss: 30.666325	CCC: 0.6665

Test set: Average loss: 5.7724, CCC: -0.0515185780823

Train Epoch: 5 [0/9453 (0%)]	Loss: 38.093788	CCC: -0.1149

Test set: Average loss: 5.2238, CCC: -0.571395039558

Train Epoch: 6 [0/9453 (0%)]	Loss: 24.294964	CCC: 3.5380

Test set: Average loss: 7.4937, CCC: -0.413353234529

Train Epoch: 7 [0/9453 (0%)]	Loss: 36.731014	CCC: 7.5798

Test set: Average loss: 6.6695, CCC: 0.184267744422

Train Epoch: 8 [0/9453 (0%)]	Loss: 43.764099	CCC: 0.2880

Test set: Average loss: 5.2322, CCC: -3.47580528259

Train Epoch: 9 [0/9453 (0%)]	Loss: 41.720760	CCC: 2.2796

Test set: Average loss: 4.5768, CCC: -0.6231442